In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

Wczytanie danych:

In [2]:
y_train=pd.read_csv('y_train.csv')
y_test=pd.read_csv('y_test.csv')
X_train=pd.read_csv('X_train.csv')
X_test=pd.read_csv('X_test.csv')

In [3]:
X_train

,sex,cabin,CabinReduced,cabin_map,cabin_map_reduced,sex_map
0,female,NaN,NaN,6,5,0
1,female,NaN,NaN,6,5,0
2,female,NaN,NaN,6,5,0
3,male,NaN,NaN,6,5,1
4,female,NaN,NaN,6,5,0
...,...,...,...,...,...,...
911,female,NaN,NaN,6,5,0
912,male,NaN,NaN,6,5,1
913,female,NaN,NaN,6,5,0
914,female,NaN,NaN,6,5,0


In [85]:
roc_train_high=[]
roc_test_high=[]
roc_train_low=[]
roc_test_low=[]

def roc_auc_high(y_test,PredTest,y_train,PredTrain):
    roc_test_high.append(roc_auc_score(y_test,PredTest))
    roc_train_high.append(roc_auc_score(y_train,PredTrain))
    print('Dane testowe: {}'.format(roc_train_high[0]))
    print('Dane treningowe: {}'.format(roc_test_high[0]))

def roc_auc_low(y_test,PredTest,y_train,PredTrain):
    roc_test_low.append(roc_auc_score(y_test,PredTest))
    roc_train_low.append(roc_auc_score(y_train,PredTrain))
    print('Dane testowe: {}'.format(roc_train_low[0]))
    print('Dane treningowe: {}'.format(roc_test_low[0]))

<h2> Random Forest<h2>

**Algorytm Random Forests** jest to estymator który dopasowuje wiele drzew decyzyjnych do próbek pobranych z badanego zbioru danych.
Przy pomocy uśredniania steruje dokładnością predykcji i kontroluje zjawisko over-fittingu. 

RandomForestClassifier, parametry:
- n_estimators - liczba drzew w lesie
- random state - kontroluje losowość procesu bootstrapingu 

**Dane o dużej liczebności cech:**

In [86]:
#Tworzenie modelu
RF = RandomForestClassifier(n_estimators=220,random_state=42)

#Trenowanie modelu
RF.fit(X_train[['sex_map','cabin_map']],np.ravel(y_train))

#Predykcja
PredTrain = RF.predict_proba(X_train[['sex_map','cabin_map']])
PredTest = RF.predict_proba(X_test[['sex_map','cabin_map']].fillna(0))



In [87]:
roc_auc_high(y_test,PredTest[:, 1],y_train,PredTrain[:, 1])

Dane testowe: 0.8529126140314685
Dane treningowe: 0.7627758420441345


**Dane o małej liczebności cech:**

In [88]:
#Trenowanie modelu
RF.fit(X_train[['sex_map','cabin_map']],np.ravel(y_train))

#Predykcja
PredTrain = RF.predict_proba(X_train[['sex_map','cabin_map_reduced']])
PredTest = RF.predict_proba(X_test[['sex_map','cabin_map_reduced']].fillna(0))

In [89]:
roc_auc_low(y_test,PredTest[:, 1],y_train,PredTrain[:, 1])

Dane testowe: 0.7334544301822994
Dane treningowe: 0.7017449256125214


W przypadku algorytmu Random Forests dane o dużej liczebności cech uzyskały lepsze wyniki niż dane o małej liczebności, ponieważ przez dużą kardynalnośc cech nastąpiło zjawisko overfittingu -> wynik dla danych treningowych (0,85) zauważalnie lepszy niż dla danych testowych (0,76).

<h2> Logistic Regression<h2>

Regresja logistyczna co do zasady jest podobna do zwykłej regresji liniowej. Podstawową różnicą jest to, że regresja logistyczna nie przewiduje wartości "y", tylko przewidywane jest prawdopodobieństwo wystąpienia danej wartości dla zmiennej "y". Stosowana jest tutaj funkcja logistyczna postaci: f(x)=1/[1+e^(-Lx)], która wymnożona przez dowolną liczbę rzeczywistą daje wynik w przedziale 0-1, a więc wspomniane wcześniej prawdopodobieństwo. Na koniec, na podstawie prawdopodobieństwa następuje przypisanie danej obserwacji do danej klasy.

**Dane o małej liczebności cech**

In [90]:
#Tworzenie modelu
LR = LogisticRegression(random_state=42)

#Trenowanie modelu
LR.fit(X_train[['sex_map','cabin_map_reduced']],np.ravel(y_train))

#Predykcja
PredTrain = LR.predict_proba(X_train[['sex_map','cabin_map_reduced']])
PredTest = LR.predict_proba(X_test[['sex_map','cabin_map_reduced']].fillna(0))

In [91]:
roc_auc_low(y_test,PredTest[:, 1],y_train,PredTrain[:, 1])

Dane testowe: 0.7334544301822994
Dane treningowe: 0.7017449256125214



**Dane o dużej liczebności cech**

In [92]:
#Trenowanie modelu
LR.fit(X_train[['sex_map','cabin_map']],np.ravel(y_train))

#Predykcja
PredTrain = LR.predict_proba(X_train[['sex_map','cabin_map']])
PredTest = LR.predict_proba(X_test[['sex_map','cabin_map']].fillna(0))

In [93]:
roc_auc_high(y_test,PredTest[:, 1],y_train,PredTrain[:, 1])

Dane testowe: 0.8529126140314685
Dane treningowe: 0.7627758420441345


W przypadku algorytmu LogisticReggresion wyniki dla obu zbiorów, o dużej i małej kardynalności, są dla siebie zbliżone, przy czym dla zbioru o małej liczebności cech różnica pomiędzy zbiorem testowym i treningowym jest minimalna i wynosi jedynie 0.0004.

<h2> Gradient Boosting Classifier<h2>

Gradient Boosting Classifier jest to algorytm skłądający się z N drzew decyzyjnych. W pierwszym kroku następuje predykcja wartości "y" zbioru treningowego przy pomocy wartości "X" tego zbioru. Następnie obliczana jest różnica pomiędzy predykcją a danymi rzeczywistymi. Różnica ta jest wykorzystana w kolejnym kroku jako zmienna objaśniana, przy czym dodatkowo zostaje przemnożona przez wartości współczynnika "learning_rate" i ponownie następuje predykcja itd. Innymi słowy każde kolejne drzewo decyzyjne dąży do zminimalizowania błędów popełnionych przez swojego poprzednika.

**Dane o dużej liczebności cech**

In [94]:
#Tworzenie modelu
GBC = GradientBoostingClassifier(n_estimators =220, random_state=42)

#Trenowanie modelu
GBC.fit(X_train[['sex_map','cabin_map']],np.ravel(y_train))

#Predykcja
PredTrain = GBC.predict_proba(X_train[['sex_map','cabin_map']])
PredTest = GBC.predict_proba(X_test[['sex_map','cabin_map']].fillna(0))

In [95]:
roc_auc_high(y_test,PredTest[:, 1],y_train,PredTrain[:, 1])

Dane testowe: 0.8529126140314685
Dane treningowe: 0.7627758420441345


**Dane o małej liczebości cech**

In [96]:
#Trenowanie modelu
GBC.fit(X_train[['sex_map','cabin_map_reduced']],np.ravel(y_train))

#Predykcja
PredTrain = GBC.predict_proba(X_train[['sex_map','cabin_map_reduced']])
PredTest = GBC.predict_proba(X_test[['sex_map','cabin_map_reduced']].fillna(0))

In [97]:
roc_auc_low(y_test,PredTest[:, 1],y_train,PredTrain[:, 1])

Dane testowe: 0.7334544301822994
Dane treningowe: 0.7017449256125214


Dla Gradient Boosting Classifier Dane o dużej liczebności cech wskazują na występowanie zjawiska overfittingu (dane treningowe - 0.86, dane testowe - 0.75). Dla danych o małej kardynalności różnica pomiędzy zbiorami wynosi 0.01.

<h2> Ada Boost Classifier<h2>

Ada Boost Clasifier jest podobny do poprzedniego algorytmu. Również wykonuje sekwencję drzew decyzyjnych starająch się minimalizować błędy popełnione przez swojego poprzednika. Jednak różnica polega na sposobie uwzględniania błędów poprzednika. Na podstawie wyliczonego błędu są ustanowione wagi dla zbioru treningowego. Obserwacje które miały największe odchyłki pomiędzy predykcją a rzeczywistą wartością w kolejnym drzewie dostają wyższe wagi niż obserwacje prawdidłowo przewidziane. W ten sposób każde kolejne drzewo coraz lepiej dopasowuje się do danych.

**Dane o dużej liczebności**

In [98]:
#Tworzenie modelu
ADC = AdaBoostClassifier(n_estimators =220, random_state=42)

#Trenowanie modelu
ADC.fit(X_train[['sex_map','cabin_map']],np.ravel(y_train))

#Predykcja
PredTrain = ADC.predict_proba(X_train[['sex_map','cabin_map']])
PredTest = ADC.predict_proba(X_test[['sex_map','cabin_map']].fillna(0))

In [99]:
roc_auc_high(y_test,PredTest[:, 1],y_train,PredTrain[:, 1])

Dane testowe: 0.8529126140314685
Dane treningowe: 0.7627758420441345


**Dane o małej liczebnośći**

In [100]:
#Trenowanie modelu
ADC.fit(X_train[['sex_map','cabin_map_reduced']],np.ravel(y_train))

#Predykcja
PredTrain = ADC.predict_proba(X_train[['sex_map','cabin_map_reduced']])
PredTest = ADC.predict_proba(X_test[['sex_map','cabin_map_reduced']].fillna(0))

In [101]:
roc_auc_low(y_test,PredTest[:, 1],y_train,PredTrain[:, 1])

Dane testowe: 0.7334544301822994
Dane treningowe: 0.7017449256125214


Dla AdaBoost Classifier Dane o dużej liczebności cech wskazują na występowanie zjawiska overfittingu (dane treningowe - 0.83, dane testowe - 0.77). Wynik dla danych o małej kardynalności różnica pomiędzy zbiorami wynosi 0.016.

In [102]:
data={
    'roc_test_low':roc_test_low,
    'roc_train_low':roc_train_low,
    'roc_test_high':roc_test_high,
    'roc_train_high':roc_train_high
    }

df=pd.DataFrame(data,index=['Random Forest','Logistic Regression','Gradient Boosting Classifier','AdaBoost Classifier'])

In [108]:
df['diff_low']=df['roc_train_low']-df['roc_test_low']
df['diff_high']=df['roc_train_high']-df['roc_test_high']
df=df[['roc_test_low','roc_train_low','diff_low','roc_test_high','roc_train_high','diff_high']]
df.round(4)

,roc_test_low,roc_train_low,diff_low,roc_test_high,roc_train_high,diff_high
Random Forest,0.7017,0.7335,0.0317,0.7628,0.8529,0.0901
Logistic Regression,0.8023,0.8027,0.0004,0.7900,0.8122,0.0222
Gradient Boosting Classifier,0.8013,0.8167,0.0154,0.7562,0.8604,0.1043
AdaBoost Classifier,0.7999,0.8162,0.0162,0.7777,0.8331,0.0554


<h1> Podsumowanie<h1>

Dla każdego z algorytmów w przypadku danych o dużej liczebności można zauważyc zjawisko overfittingu, najwyraźniej jest ono widoczne dla  Gradient Boost Classifier i Random Forest. Dla danych o niskiej kardynalności różnice pomiędzy zbiorem testowym i treningowym są zauważalnie mniejsze niż w przypadku danych o dużej kardynalności. Moim zdaniem najlepsze rezultaty daje algorytm Logistic Reggresion, ponieważ osiąga najwyższą wartość statystyki roc_auc_score dla danych o małej kardynalności.

<h1> predict vs predict_proba<h1>

<b>predict()</b>  - zwraca jako wynik dokładne dopasowanie do danej klasy <br>
<b>predict_proba()</b> - zwraca jako wynik klasy prawdopodobieństwa, dla każdej potencjalnej klasy jest obliczane prawdopodobieństwo zaliczenia do niej danego elementu